In [21]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

In [15]:
# ================================================================== #
#                 1. Basic autograd example 1                        #
# ================================================================== #

In [36]:
# Create tensors.
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph.
y = w * x + b    # y = 2 * x + 3

# Compute gradients.
y.backward()

# Print out the gradients.
print("x.grad: ", x.grad)    # x.grad = 2 
print("w.grad: ", w.grad)    # w.grad = 1 
print("b.grad: ", b.grad)    # b.grad = 1 

x.grad:  tensor(2.)
w.grad:  tensor(1.)
b.grad:  tensor(1.)


In [16]:
# ================================================================== #
#                 2. Basic autograd example 2                        #
# ================================================================== #

In [38]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)
print ('w: ', linear.weight)
print ('b: ', linear.bias, "\n")


# Build loss function and optimizer.
criterion = nn.L1Loss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)


# Forward pass.
pred = linear(x)

# Compute loss.
loss = criterion(pred, y)
print('loss: ', loss.item(), "\n")

# Backward pass.
loss.backward()

# Print out the gradients.
print ('dL/dw: ', linear.weight.grad) 
print ('dL/db: ', linear.bias.grad, "\n")


# 1-step gradient descent.
optimizer.step()


# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = criterion(pred, y)
print('loss after 1 step optimization: ', loss.item(), "\n")

w:  Parameter containing:
tensor([[ 0.4363, -0.4807,  0.1703],
        [ 0.3007,  0.4390,  0.5343]], requires_grad=True)
b:  Parameter containing:
tensor([0.5669, 0.1098], requires_grad=True) 

loss:  0.9940336346626282 

dL/dw:  tensor([[ 0.1265, -0.0178,  0.1408],
        [-0.0044,  0.2599,  0.0548]])
dL/db:  tensor([ 0.1000, -0.2000]) 

loss after 1 step optimization:  0.9924662709236145 



#### usefull functions

In [ ]:
# Dataset Preparation
import torchvision.transforms as tfs
data_tfs = tfs.Compose([
  tfs.ToTensor(),
  tfs.Normalize((0.5), (0.5))]) # Compose совмещает преобразования указанные в списке

from torch.utils.data import Dataset
# create a dataset from our data

from torch.utils.data import DataLoader
train_loader = DataLoader(train, batch_size=batch_size, drop_last=True, shuffle=True, num_workers=2)
test_loader = DataLoader(test, batch_size=batch_size, drop_last=True, shuffle=False, num_workers=2)

In [ ]:
from torchsummary import summary
summary(model, (features,), batch_size=228) # model summary 

In [ ]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.99))

In [1]:
# Формула выхода после свертки
# (Input_W - KernelSize)//Stride + 1

In [ ]:
# Использование Sequential для создания сети
two_layer_net = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
    torch.nn.Softmax())

#### model example

In [ ]:
# Использование модулей для создания сети 
# nn.ModuleList или nn.ModuleDict
# nn.Parameter, nn.ParameterList или nn.ParameterDict 

class MyConvNet(nn.Module):
    def __init__(self):
        # вызов конструктора предка
        super().__init__()
        # необходмо заранее знать, сколько каналов у картинки (сейчас = 1),
        # которую будем подавать в сеть, больше ничего
        # про входящие картинки знать не нужно
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=5)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.fc1 = nn.Linear(6 * 6 * 128, 128)  # !!!
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
        self.my_useless_bias = nn.Parameter(torch.ones(1, H, requires_grad=True))
        self.more_of_my_useless_biases = nn.ParameterList([
            nn.Parameter(torch.ones(1, H, requires_grad=True)),
            nn.Parameter(torch.ones(1, H, requires_grad=True))])

    def forward(self, x):
        #(32,32)
        x = self.pool(F.relu(self.conv1(x)))
        #(14,14)
        x = self.pool(F.relu(self.conv2(x)))
        #(6,6)
        #print(x.shape)
        x = x.view(-1, 6 * 6 * 128)  # !!!
        x = F.relu(self.fc1(x))
        for b in self.more_of_my_useless_biases:
            x += b
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x += self.my_useless_bias
        return x

In [ ]:
# пример взят из официального туториала: 
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

net = MyConvNet().to(device)

loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

# итерируемся
for epoch in tqdm_notebook(range(10)):

    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(trainloader)):
        # так получаем текущий батч
        X_batch, y_batch = batch

        # обнуляем веса
        optimizer.zero_grad()

        # forward + backward + optimize
        y_pred = net(X_batch.to(device))
        loss = loss_fn(y_pred, y_batch.to(device))
        loss.backward()
        optimizer.step()

        # выведем текущий loss
        running_loss += loss.item()
        # выводем качество каждые 2000 батчей
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Обучение закончено')

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
    for data in testloader:
        images, labels = data
        y_pred = net(images.to(device))#.view(4, -1))
        _, predicted = torch.max(y_pred, 1)
        c = (predicted.cpu().detach() == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))